In [1]:
#load packages I need
import os
import tools
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import pyscenic
from pyscenic.aucell import aucell

sc.logging.print_header()

scanpy==1.10.3 anndata==0.10.9 umap==0.5.7 numpy==1.23.5 scipy==1.14.1 pandas==2.2.3 scikit-learn==1.6.0 statsmodels==0.14.4 pynndescent==0.5.13


In [2]:
#show current directory 
os.getcwd()
os.chdir('/scratch/user/s4436039/scdata/Python_Integration_Sep')
os.getcwd()

'/scratch/user/s4436039/scdata/Python_Integration_Sep'

In [3]:
# read in data
data = sc.read_h5ad('NRclean_clustered2.h5ad')

In [4]:
#make two versions of the data, once containing just cDC1 and mregDC, and another containing just cDC2 and mregDC
data_DC1mreg = data[data.obs["NR_annotations_simple"].isin(["cDC1","mregDC"])]
data_DC2mreg = data[data.obs["NR_annotations_simple"].isin(["cDC2","mregDC"])]

# pyscenic:

## 1. GRN Inference (gene regulatory network)

# following online tute:

In [2]:
os.chdir('/scratch/user/s4436039/scdata/PySCENIC')
os.getcwd()

'/scratch/user/s4436039/scdata/PySCENIC'

In [3]:
# transcription factors list
f_tfs = "allTFs_hg38.txt" 

In [9]:
#make expression matrix
expr_matrix = pd.DataFrame(data_DC1mreg.X, index=data_DC1mreg.obs_names, columns=data_DC1mreg.var_names)

In [12]:
expr_matrix.to_csv('DC1mreg_expr_matrix.tsv', sep='\t', index=False)

In [4]:
f_expr = "DC1mreg_expr_matrix.tsv" 

In [10]:
import loompy as lp
import numpy as np

In [7]:
# Save the path where you want loom file to save (current directory)
f_loom_path_DC1mreg = "DC1mreg_loom.loom"

In [7]:
# row_attrs = { 
#     "Gene": np.array(data_DC1mreg.var.index) ,
# }
# col_attrs = { 
#     "CellID":  np.array(data_DC1mreg.obs.index) ,
#     "nGene": np.array( np.sum(data_DC1mreg.X.transpose()>0 , axis=0)).flatten() ,
#     "nUMI": np.array( np.sum(data_DC1mreg.X.transpose() , axis=0)).flatten() ,
# }

# lp.create( f_loom_path_DC1mreg, data_DC1mreg.X.transpose(), row_attrs, col_attrs )

In [5]:
!arboreto_with_multiprocessing.py DC1mreg_loom.loom allTFs_hg38.txt --output adj.tsv --num_workers 1

Traceback (most recent call last):
  File "/home/s4436039/miniforge3/envs/pyscenic/lib/python3.10/site-packages/dask/dataframe/__init__.py", line 4, in <module>
    from .core import (DataFrame, Series, Index, _Frame, map_partitions,
  File "/home/s4436039/miniforge3/envs/pyscenic/lib/python3.10/site-packages/dask/dataframe/core.py", line 35, in <module>
    from . import methods
  File "/home/s4436039/miniforge3/envs/pyscenic/lib/python3.10/site-packages/dask/dataframe/methods.py", line 10, in <module>
    from .utils import PANDAS_VERSION
  File "/home/s4436039/miniforge3/envs/pyscenic/lib/python3.10/site-packages/dask/dataframe/utils.py", line 13, in <module>
    import pandas.util.testing as tm
ModuleNotFoundError: No module named 'pandas.util.testing'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/s4436039/miniforge3/envs/pyscenic/bin/pyscenic", line 5, in <module>
    from pyscenic.cli.pyscenic import main
  

In [14]:
!pyscenic grn {f_expr} {f_tfs} -o adj.csv --num_workers 20

Traceback (most recent call last):
  File "/home/s4436039/miniforge3/envs/env/bin/pyscenic", line 5, in <module>
    from pyscenic.cli.pyscenic import main
  File "/home/s4436039/miniforge3/envs/env/lib/python3.12/site-packages/pyscenic/cli/pyscenic.py", line 19, in <module>
    from arboreto.algo import genie3, grnboost2
  File "/home/s4436039/miniforge3/envs/env/lib/python3.12/site-packages/arboreto/algo.py", line 5, in <module>
    import pandas as pd
  File "/home/s4436039/miniforge3/envs/env/lib/python3.12/site-packages/pandas/__init__.py", line 49, in <module>
    from pandas.core.api import (
  File "/home/s4436039/miniforge3/envs/env/lib/python3.12/site-packages/pandas/core/api.py", line 47, in <module>
    from pandas.core.groupby import (
  File "/home/s4436039/miniforge3/envs/env/lib/python3.12/site-packages/pandas/core/groupby/__init__.py", line 1, in <module>
    from pandas.core.groupby.generic import (
  File "/home/s4436039/miniforge3/envs/env/lib/python3.12/site-packag

## 2. Motif enrichment and pruning the GRN (based on motif enrichment)
* to perform pruning need motif rankings and TF lists

## 3. Scoring the cells (using AUCell to determine activity)

# other

# using pyscenic to extract the regulons
* downloaded necessary databases: https://pyscenic.readthedocs.io/en/latest/installation.html
* database ranking the genome --> used Homo sapiens - hg38 - refseq_r80 - SCENIC+ databases - Gene based --> hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather
* Motif to TF annotations --> 

In [12]:
# using pyscenic to extract the regulons